In [ ]:
import re
import torch
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D,Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Specify the file or folder path
# file_path = '/content/drive/MyDrive/Comppetion/train.xlsx'
# test_path="/content/drive/MyDrive/Comppetion/test .csv"

file_path="/content/drive/MyDrive/NN'23_arabic_sentiment_analysis_NLP/train.xlsx"
test_path="/content/drive/MyDrive/NN'23_arabic_sentiment_analysis_NLP/test .csv"

Mounted at /content/drive


In [ ]:
# Create Data Frame
data = pd.read_excel(file_path)

In [ ]:
print(f"Data Shape: {data.shape}")
print(f"\nFirst 6 columns in data:\n {data.head(6)}")

Data Shape: (32036, 2)

First 6 columns in data:
                                   review_description  rating
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...      -1
1  خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...       1
2  تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...      -1
3  فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...       1
4  سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...      -1
5  قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...       0


In [ ]:
# Remove duplicated
print(data.review_description.duplicated().sum())
data.drop(data[data.review_description.duplicated() == True].index, axis = 0, inplace = True)

726


In [ ]:
emojis = {
    "🙂":"يبتسم",
    "😂":"يضحك",
    "💔":"قلب حزين",
    "❤️":"حب",
    "❤":"حب",
    "😍":"حب",
    "😭":"يبكي",
    "😢":"حزن",
    "😔":"حزن",
    "♥":"حب",
    "💜":"حب",
    "😅":"يضحك",
    "🙁":"حزين",
    "💕":"حب",
    "💙":"حب",
    "😞":"حزين",
    "😊":"سعادة",
    "👏":"يصفق",
    "👌":"احسنت",
    "😴":"ينام",
    "😀":"يضحك",
    "😌":"حزين",
    "🌹":"وردة",
    "🙈":"حب",
    "😄":"يضحك",
    "😐":"محايد",
    "✌":"منتصر",
    "✨":"نجمه",
    "🤔":"تفكير",
    "😏":"يستهزء",
    "😒":"يستهزء",
    "🙄":"ملل",
    "😕":"عصبية",
    "😃":"يضحك",
    "🌸":"وردة",
    "😓":"حزن",
    "💞":"حب",
    "💗":"حب",
    "😑":"منزعج",
    "💭":"تفكير",
    "😎":"ثقة",
    "💛":"حب",
    "😩":"حزين",
    "💪":"عضلات",
    "👍":"موافق",
    "🙏🏻":"رجاء طلب",
    "😳":"مصدوم",
    "👏🏼":"تصفيق",
    "🎶":"موسيقي",
    "🌚":"صمت",
    "💚":"حب",
    "🙏":"رجاء طلب",
    "💘":"حب",
    "🍃":"سلام",
    "☺":"يضحك",
    "🐸":"ضفدع",
    "😶":"مصدوم",
    "✌️":"مرح",
    "✋🏻":"توقف",
    "😉":"غمزة",
    "🌷":"حب",
    "🙃":"مبتسم",
    "😫":"حزين",
    "😨":"مصدوم",
    "🎼 ":"موسيقي",
    "🍁":"مرح",
    "🍂":"مرح",
    "💟":"حب",
    "😪":"حزن",
    "😆":"يضحك",
    "😣":"استياء",
    "☺️":"حب",
    "😱":"كارثة",
    "😁":"يضحك",
    "😖":"استياء",
    "🏃🏼":"يجري",
    "😡":"غضب",
    "🚶":"يسير",
    "🤕":"مرض",
    "‼️":"تعجب",
    "🕊":"طائر",
    "👌🏻":"احسنت",
    "❣":"حب",
    "🙊":"مصدوم",
    "💃":"سعادة مرح",
    "💃🏼":"سعادة مرح",
    "😜":"مرح",
    "👊":"ضربة",
    "😟":"استياء",
    "💖":"حب",
    "😥":"حزن",
    "🎻":"موسيقي",
    "✒":"يكتب",
    "🚶🏻":"يسير",
    "💎":"الماظ",
    "😷":"وباء مرض",
    "☝":"واحد",
    "🚬":"تدخين",
    "💐" : "ورد",
    "🌞" : "شمس",
    "👆" : "الاول",
    "⚠️" :"تحذير",
    "🤗" : "احتواء",
    "✖️": "غلط",
    "📍"  : "مكان",
    "👸" : "ملكه",
    "👑" : "تاج",
    "✔️" : "صح",
    "💌": "قلب",
    "😲" : "مندهش",
    "💦": "ماء",
    "🚫" : "خطا",
    "👏🏻" : "برافو",
    "🏊" :"يسبح",
    "👍🏻": "تمام",
    "⭕️" :"دائره كبيره",
    "🎷" : "ساكسفون",
    "👋": "تلويح باليد",
    "✌🏼": "علامه النصر",
    "🌝":"مبتسم",
    "➿"  : "عقده مزدوجه",
    "💪🏼" : "قوي",
    "📩":  "تواصل معي",
    "☕️": "قهوه",
    "😧" : "قلق و صدمة",
    "🗨": "رسالة",
    "❗️" :"تعجب",
    "🙆🏻": "اشاره موافقه",
    "👯" :"اخوات",
    "©" :  "رمز",
    "👵🏽" :"سيده عجوزه",
    "🐣": "كتكوت",
    "🙌": "تشجيع",
    "🙇": "شخص ينحني",
    "👐🏽":"ايدي مفتوحه",
    "👌🏽": "بالظبط",
    "⁉️" : "استنكار",
    "⚽️": "كوره",
    "🕶" :"حب",
    "🎈" :"بالون",
    "🎀":    "ورده",
    "💵":  "فلوس",
    "😋":  "جائع",
    "😛":  "يغيظ",
    "😠":  "غاضب",
    "✍🏻":  "يكتب",
    "🌾":  "ارز",
    "👣":  "اثر قدمين",
    "❌":"رفض",
    "🍟":"طعام",
    "👬":"صداقة",
    "🐰":"ارنب",
    "☂":"مطر",
    "⚜":"مملكة فرنسا",
    "🐑":"خروف",
    "🗣":"صوت مرتفع",
    "👌🏼":"احسنت",
    "☘":"مرح",
    "😮":"صدمة",
    "😦":"قلق",
    "⭕":"الحق",
    "✏️":"قلم",
    "ℹ":"معلومات",
    "🙍🏻":"رفض",
    "⚪️":"نضارة نقاء",
    "🐤":"حزن",
    "💫":"مرح",
    "💝":"حب",
    "🍔":"طعام",
    "❤︎":"حب",
    "✈️":"سفر",
    "🏃🏻‍♀️":"يسير",
    "🍳":"ذكر",
    "🎤":"مايك غناء",
    "🎾":"كره",
    "🐔":"دجاجة",
    "🙋":"سؤال",
    "📮":"بحر",
    "💉":"دواء",
    "🙏🏼":"رجاء طلب",
    "💂🏿 ":"حارس",
    "🎬":"سينما",
    "♦️":"مرح",
    "💡":"قكرة",
    "‼":"تعجب",
    "👼":"طفل",
    "🔑":"مفتاح",
    "♥️":"حب",
    "🕋":"كعبة",
    "🐓":"دجاجة",
    "💩":"معترض",
    "👽":"فضائي",
    "☔️":"مطر",
    "🍷":"عصير",
    "🌟":"نجمة",
    "☁️":"سحب",
    "👃":"معترض",
    "🌺":"مرح",
    "🔪":"سكينة",
    "♨":"سخونية",
    "👊🏼":"ضرب",
    "✏":"قلم",
    "🚶🏾‍♀️":"يسير",
    "👊":"ضربة",
    "◾️":"وقف",
    "😚":"حب",
    "🔸":"مرح",
    "👎🏻":"لا يعجبني",
    "👊🏽":"ضربة",
    "😙":"حب",
    "🎥":"تصوير",
    "👉":"جذب انتباه",
    "👏🏽":"يصفق",
    "💪🏻":"عضلات",
    "🏴":"اسود",
    "🔥":"حريق",
    "😬":"عدم الراحة",
    "👊🏿":"يضرب",
    "🌿":"ورقه شجره",
    "✋🏼":"كف ايد",
    "👐":"ايدي مفتوحه",
    "☠️":"وجه مرعب",
    "🎉":"يهنئ",
    "🔕" :"صامت",
    "😿":"وجه حزين",
    "☹️":"وجه يائس",
    "😘" :"حب",
    "😰" :"خوف و حزن",
    "🌼":"ورده",
    "💋":  "بوسه",
    "👇":"لاسفل",
    "❣️":"حب",
    "🎧":"سماعات",
    "📝":"يكتب",
    "😇":"دايخ",
    "😈":"رعب",
    "🏃":"يجري",
    "✌🏻":"علامه النصر",
    "🔫":"يضرب",
    "❗️":"تعجب",
    "👎":"غير موافق",
    "🔐":"قفل",
    "👈":"لليمين",
    "™":"رمز",
    "🚶🏽":"يتمشي",
    "😯":"متفاجأ",
    "✊":"يد مغلقه",
    "😻":"اعجاب",
    "🙉" :"قرد",
    "👧":"طفله صغيره",
    "🔴":"دائره حمراء",
    "💪🏽":"قوه",
    "💤":"ينام",
    "👀":"ينظر",
    "✍🏻":"يكتب",
    "❄️":"تلج",
    "💀":"رعب",
    "😤":"وجه عابس",
    "🖋":"قلم",
    "🎩":"كاب",
    "☕️":"قهوه",
    "😹":"ضحك",
    "💓":"حب",
    "☄️ ":"نار",
    "👻":"رعب",
    "❎":"خطء",
    "🤮":"حزن",
    '🏻':"احمر",
    "😝":"مرح"
}

In [ ]:
import re

arabic_puncituation = ['،','؟']
def remove_character(word):
  for char in arabic_puncituation:
    if char in word:
        word = word.replace(char, '')
  return word


def remove_character(word):
    for char in arabic_puncituation:
        if char in word:
            word = word.replace(char, '')  # Removes the specified character from the word
    return word

number = ['٠', '١', '٢', '٣', '٤', '٥', '٦', '٧', '٨', '٩']
def contains_any_number(word):
  for char in number:
      if char in word:
          return False
  return True

def is_arabic_word(word):
    arabic_pattern = re.compile(r'^[؀-ۿ ً-ْ]+$')  # Arabic Unicode range
    return bool(arabic_pattern.match(word))


def checkemojie(text):
    result_text = ""
    for char in text:
        if char in emojis:
            result_text += " " + emojis[char] + " "
        else:
            result_text += char
    return result_text

def remove_stopwords_and_punctuation(text):
    stop_words = set(stopwords.words('arabic'))

    words = checkemojie(text)
    words = words.split()
    filtered_words = [word for word in words if word not in stop_words and word not in string.punctuation and is_arabic_word(word) == True and contains_any_number(word) == True]
    modified_list = [remove_character(word) for word in filtered_words]

    return ' '.join(modified_list)

In [ ]:
import nltk
import string
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('snowball_data')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


print(data.head(10))

data['Token'] = data['review_description'].apply(remove_stopwords_and_punctuation)


print(data.head(10))

print(data.iloc[19])


[nltk_data] Downloading package snowball_data to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


                                  review_description  rating
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...      -1
1  خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...       1
2  تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...      -1
3  فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...       1
4  سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...      -1
5  قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...       0
6                                         احلئ تطبيق       1
7                                      رائع واو مدهش       1
8  مکو بس البحرین وعمان وغیرهه بس العراق مکو یعنی...      -1
9                    تطبيق جميل يستاهل الخمس نجوم👍👍👍       1
                                  review_description  rating  \
0  شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...      -1   
1  خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...       1   
2  تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...      -1   
3  فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...       1   
4  سيء جد

In [ ]:
print(data.iloc[9])

print(data.iloc[9]['Token'])


review_description                   تطبيق جميل يستاهل الخمس نجوم👍👍👍
rating                                                             1
Token                 تطبيق جميل يستاهل الخمس نجوم موافق موافق موافق
Name: 9, dtype: object
تطبيق جميل يستاهل الخمس نجوم موافق موافق موافق


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Create a tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Token'])

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(data['Token'])

# Padding sequences to a maximum length
max_length =  max(map(len, sequences))  # Define the maximum sequence length
print(max_length)

padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

print("Word Index:\n", tokenizer.word_index)
print("\nSequences:\n", sequences)
print("\nPadded Sequences:\n", padded_sequences)


286
Word Index:
 {'حب': 1, 'جدا': 2, 'تطبيق': 3, 'التطبيق': 4, 'موافق': 5, 'ممتاز': 6, 'الطلب': 7, 'برنامج': 8, 'البرنامج': 9, 'انا': 10, 'مش': 11, 'جميل': 12, 'المطاعم': 13, 'التوصيل': 14, 'رائع': 15, 'طلبات': 16, 'حلو': 17, 'الله': 18, 'خدمة': 19, 'توصيل': 20, 'يضحك': 21, 'ان': 22, 'المطعم': 23, 'مره': 24, 'والله': 25, 'اطلب': 26, 'افضل': 27, 'طلبت': 28, 'فاشل': 29, 'جيد': 30, 'او': 31, 'علي': 32, 'شي': 33, 'الي': 34, 'اي': 35, 'شكرا': 36, 'اكثر': 37, 'انصح': 38, 'غضب': 39, 'يوصل': 40, 'خدمه': 41, 'سيء': 42, 'العملاء': 43, 'حزن': 44, 'مطعم': 45, 'طلب': 46, 'فى': 47, 'يوجد': 48, 'مطاعم': 49, 'يكون': 50, 'انه': 51, 'اللي': 52, 'غاضب': 53, 'احسن': 54, 'يعني': 55, 'احسنت': 56, 'جداً': 57, 'تم': 58, 'عشان': 59, 'سيئ': 60, 'مو': 61, 'ليش': 62, 'مرة': 63, 'ليه': 64, 'سريع': 65, 'ممتازة': 66, 'الوقت': 67, 'سئ': 68, 'خصم': 69, 'روعه': 70, 'يتم': 71, 'الطلبات': 72, 'الاكل': 73, 'اول': 74, 'شركة': 75, 'وقت': 76, 'موقع': 77, 'ممتازه': 78, 'الموقع': 79, 'زي': 80, 'ارجو': 81, 'وانا': 82, 'عندي': 8

In [ ]:
padded_sequences = pd.DataFrame(padded_sequences)


In [ ]:
print(len(tokenizer.word_index))


38081


In [ ]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data['rating'])
print(f"Encoded Labels:\n {encoded_labels}")

Encoded Labels:
 [0 2 0 ... 0 0 2]


# **Model**

# **Transformer**

In [ ]:
import re
import string
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Embedding, GlobalAveragePooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.utils import resample

def transformer_classification_model_v2(max_seq_length, vocab_size, num_heads=8, ff_dim=32, num_blocks=6, dropout_rate=0.1):
    inputs = Input(shape=(max_seq_length,))

    embedding_layer = Embedding(input_dim=vocab_size, output_dim=128)(inputs)

    x = embedding_layer
    for _ in range(num_blocks):
        x = transformer_block(x, max_seq_length, num_heads, ff_dim, dropout_rate)

    x = GlobalAveragePooling1D()(x)

    # Dense layers for multi classification
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    outputs = Dense(len(np.unique(encoded_labels)), activation='softmax')(x)  # Assuming binary classification, adjust for multiclass
    model = Model(inputs=inputs, outputs=outputs)

    return model

def transformer_block(x, max_seq_length, num_heads, ff_dim, dropout_rate=0.1):
    # Multi-Head Self Attention
    multi_head_attention = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=256, dropout=dropout_rate
    )(x, x)

    # Add & Norm
    x = tf.keras.layers.Add()([x, multi_head_attention])
    x = tf.keras.layers.LayerNormalization()(x)

    # Feed Forward Part
    ff = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(x)
    ff = tf.keras.layers.Dropout(dropout_rate)(ff)
    ff = tf.keras.layers.Conv1D(filters=128, kernel_size=1)(ff)

    # Add & Norm
    x = tf.keras.layers.Add()([x, ff])
    x = tf.keras.layers.LayerNormalization()(x)

    return x

# Example usage:
max_seq_length = 286
vocab_size = len(tokenizer.word_index) + 1
num_heads = 8
ff_dim = 50
num_blocks = 2
dropout_rate = 0.1

# Use the updated model
model = transformer_classification_model_v2(max_seq_length, vocab_size, num_heads, ff_dim, num_blocks, dropout_rate)
def lr_schedule(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * 0.9  # Adjust the multiplier as needed

optimizer = Adam(learning_rate=0.001)  # Set an initial learning rate
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with learning rate scheduling
model.fit(padded_sequences, encoded_labels, validation_split=0.2, epochs=15, batch_size=150, callbacks=[early_stopping, LearningRateScheduler(lr_schedule)])



Epoch 1/15
167/167 [==============================] - 132s 759ms/step - loss: 0.8380 - accuracy: 0.5988 - val_loss: 0.7005 - val_accuracy: 0.7391 - lr: 0.0010
Epoch 2/15
167/167 [==============================] - 124s 744ms/step - loss: 0.5864 - accuracy: 0.7777 - val_loss: 0.5622 - val_accuracy: 0.7876 - lr: 0.0010
Epoch 3/15
167/167 [==============================] - 125s 750ms/step - loss: 0.4326 - accuracy: 0.8487 - val_loss: 0.5332 - val_accuracy: 0.8007 - lr: 0.0010
Epoch 4/15
167/167 [==============================] - 125s 749ms/step - loss: 0.3593 - accuracy: 0.8758 - val_loss: 0.5886 - val_accuracy: 0.7994 - lr: 0.0010
Epoch 5/15
167/167 [==============================] - 124s 742ms/step - loss: 0.2753 - accuracy: 0.9028 - val_loss: 0.6969 - val_accuracy: 0.7972 - lr: 0.0010
Epoch 6/15
167/167 [==============================] - 124s 744ms/step - loss: 0.2237 - accuracy: 0.9196 - val_loss: 0.7231 - val_accuracy: 0.7603 - lr: 9.0000e-04
Epoch 7/15
167/167 [======================

In [ ]:
test_path="/content/drive/MyDrive/NN'23_arabic_sentiment_analysis_NLP/test .csv"
test_data=pd.read_csv(test_path)

x_test=test_data["review_description"]
ids=test_data["ID"]
reviews_test=pd.DataFrame(x_test)
x_test
# reviews_test['review_description'], reviews_test['emojis'] = zip(*reviews_test['review_description'].apply(clean_and_extract_emojis))




0      اهنئكم على خدمه العملاء في المحادثه المباشره م...
1      ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...
2        كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء
3                                                شغل طيب
4                                             بعد ماجربت
                             ...                        
995                                                يستهل
996                               خدمة سيئة بكل المعايير
997                                        لؤي٠٣٣٢لؤ٣٤٣س
998                       تطبيق غير صادق ف خصم الكوبونات
999    تأخير + الموظفين غير مؤهلة .للأسف انا استخدم ط...
Name: review_description, Length: 1000, dtype: object

In [ ]:
x_test =x_test.apply(remove_stopwords_and_punctuation)
x_test

0      اهنئكم خدمه العملاء المحادثه المباشره قصرو الل...
1      ممتاز جدا اتمنى ان تكون المسابقات والجوائز طلب...
2                 محملته يقول تم ايقاف عشان تسوون الخطاء
3                                                شغل طيب
4                                                 ماجربت
                             ...                        
995                                                يستهل
996                               خدمة سيئة بكل المعايير
997                                                     
998                             تطبيق صادق خصم الكوبونات
999    تأخير الموظفين مؤهلة انا استخدم طلبات سنة والي...
Name: review_description, Length: 1000, dtype: object

In [ ]:

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(x_test)


padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

print("Word Index:\n", tokenizer.word_index)
print("\nSequences:\n", sequences)
print("\nPadded Sequences:\n", padded_sequences)

x_test = pd.DataFrame(padded_sequences)



Word Index:
 {'حب': 1, 'جدا': 2, 'تطبيق': 3, 'التطبيق': 4, 'موافق': 5, 'ممتاز': 6, 'الطلب': 7, 'برنامج': 8, 'البرنامج': 9, 'انا': 10, 'مش': 11, 'جميل': 12, 'المطاعم': 13, 'التوصيل': 14, 'رائع': 15, 'طلبات': 16, 'حلو': 17, 'الله': 18, 'خدمة': 19, 'توصيل': 20, 'يضحك': 21, 'ان': 22, 'المطعم': 23, 'مره': 24, 'والله': 25, 'اطلب': 26, 'افضل': 27, 'طلبت': 28, 'فاشل': 29, 'جيد': 30, 'او': 31, 'علي': 32, 'شي': 33, 'الي': 34, 'اي': 35, 'شكرا': 36, 'اكثر': 37, 'انصح': 38, 'غضب': 39, 'يوصل': 40, 'خدمه': 41, 'سيء': 42, 'العملاء': 43, 'حزن': 44, 'مطعم': 45, 'طلب': 46, 'فى': 47, 'يوجد': 48, 'مطاعم': 49, 'يكون': 50, 'انه': 51, 'اللي': 52, 'غاضب': 53, 'احسن': 54, 'يعني': 55, 'احسنت': 56, 'جداً': 57, 'تم': 58, 'عشان': 59, 'سيئ': 60, 'مو': 61, 'ليش': 62, 'مرة': 63, 'ليه': 64, 'سريع': 65, 'ممتازة': 66, 'الوقت': 67, 'سئ': 68, 'خصم': 69, 'روعه': 70, 'يتم': 71, 'الطلبات': 72, 'الاكل': 73, 'اول': 74, 'شركة': 75, 'وقت': 76, 'موقع': 77, 'ممتازه': 78, 'الموقع': 79, 'زي': 80, 'ارجو': 81, 'وانا': 82, 'عندي': 83, '

In [ ]:
prediction=model.predict(x_test)
import numpy as np
p=np.argmax(prediction,1)
print(p)

32/32 [==============================] - 2s 52ms/step
[2 2 0 2 2 2 0 0 2 0 2 2 2 0 0 2 2 0 0 0 2 2 2 2 0 0 2 2 2 0 0 0 0 0 0 2 2
 2 2 2 2 2 2 2 2 0 2 0 2 2 2 2 0 0 0 2 0 0 0 0 2 2 0 2 2 2 0 2 0 2 0 2 2 0
 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 0 2 2 0 0 0 2 2 0 0 2 2 2
 2 2 0 2 0 2 2 2 2 2 2 2 2 2 2 2 0 2 2 0 0 2 2 2 0 2 2 2 2 0 0 2 0 2 0 0 0
 2 2 2 2 2 0 2 2 2 0 2 2 2 2 0 2 0 0 0 2 2 2 2 0 2 2 0 2 2 0 2 0 0 2 2 2 2
 2 2 2 2 2 2 0 2 2 0 2 0 0 2 2 2 2 2 0 0 2 2 2 0 2 0 0 2 0 0 2 2 0 2 2 2 0
 0 2 0 2 0 0 0 2 2 2 2 0 2 2 2 2 2 0 0 0 0 2 0 2 2 0 0 2 2 0 0 0 0 2 0 2 0
 2 2 2 2 0 2 2 0 0 0 2 0 2 0 0 0 0 0 2 2 2 2 2 2 0 0 2 2 2 2 2 2 2 2 2 2 0
 2 2 2 0 0 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 0 0 2 0 0 0 2 0 2 2 2 2 2 0 0 2 2
 2 2 2 2 2 0 2 0 2 2 2 2 0 2 2 2 2 2 0 0 2 0 2 2 2 0 0 2 0 0 0 0 2 2 0 0 0
 2 0 2 0 0 0 0 0 0 2 2 2 0 0 2 0 2 2 2 0 0 2 2 0 2 0 2 0 0 2 2 2 0 2 0 2 0
 0 2 2 0 0 0 2 2 0 0 0 2 0 2 2 0 0 2 0 2 0 0 0 0 0 2 2 0 2 0 0 0 2 0 2 2 2
 2 2 0 2 2 2 0 2 0 0 2 2 0 2 0 2 2 2 2 0 2 2 2

In [ ]:
for i in range(len(p)):
  if p[i]==0:
    p[i]=-1
  if p[i]==1:
    p[i]=0
  if p[i]==2:
    p[i]=1
    print(p)

In [ ]:
submission=pd.DataFrame()
submission['ID']=ids
submission['rating']=p
submission.to_csv('pred.csv',index=False)